https://cran.r-project.org/web/packages/stargazer/vignettes/stargazer.pdf

# Libraries and data

In [2]:
suppressWarnings(suppressPackageStartupMessages(library(tidyverse)))
suppressWarnings(suppressPackageStartupMessages(library(lmtest)))
suppressWarnings(suppressPackageStartupMessages(library(sandwich)))
suppressWarnings(suppressPackageStartupMessages(library(stargazer)))

In [3]:
grouped_df <- read_csv("grouped_results.csv", show_col_types = FALSE)

head(grouped_df)

doc_id,people,common_will,elite,gruppoP_recoded,year,perc_of_populist_toks,standardized_perc_of_populist_toks,legislature,Gov_memb,⋯,logit_left_right,classic_economic,ratio_economic,logit_economic,classic_gal_tan,ratio_gal_tan,logit_gal_tan,classic_economic_gal_tan,ratio_economic_gal_tan,logit_economic_gal_tan
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1994.AN,431,29,184,AN,1994,0.003117738,0.0591211,12,1,⋯,0.4777925,-2.20,-0.5555556,-0.9532793,-3.0800000,-1.0000000,-1.9685100,-3.0800000,-1.0000000,-1.9685100
1995.AN,905,110,318,AN,1995,0.003808147,0.5967706,12,0,⋯,0.3836704,2.15,0.3323029,0.5923709,0.0000000,NA,0.0000000,0.5400000,1.0000000,0.7323679
1996.AN,422,40,177,AN,1996,0.002344008,-0.5434142,12,0,⋯,0.3836704,2.15,0.3323029,0.5923709,0.0000000,NA,0.0000000,0.5400000,1.0000000,0.7323679
1997.AN,564,76,336,AN,1997,0.002556158,-0.3782041,13,0,⋯,0.9537010,1.52,0.1202532,0.2238034,-2.0200000,-1.0000000,-1.6174061,-2.0200000,-1.0000000,-1.6174061
1998.AN,664,49,287,AN,1998,0.002677907,-0.2833936,13,0,⋯,1.1505511,2.36,0.0779661,0.1735631,-0.7250000,-1.0000000,-0.6804883,-0.7250000,-1.0000000,-0.6804883
1999.AN,713,65,337,AN,1999,0.002788929,-0.1969357,13,0,⋯,1.5052324,6.68,0.4513513,0.9021562,0.7000001,0.1495727,0.2477382,0.7000001,0.1495727,0.2477382


# Group-level analysis

Y % tokens populisti = costant + left right + galtan + dummy governo/opposizione + effetti fissi di anno + errore

Variazioni del modello OLS: 
- diverse misure leftright e galtan (sempre a coppie simili), ricordati che ci sono anche quelle economiche 
- dummy governo opposizione con e senza supporto esterno
- errore semplice, robusto, clustered by legislature

1. constant + classic left/right + classic gal/tan + dummy governo/opposizione + effetti fissi di anno + errore
2. constant + ratio left/right + ratio gal/tan + dummy governo/opposizione + effetti fissi di anno + errore
3. constant + logit left/right + logit gal/tan + dummy governo/opposizione + effetti fissi di anno + errore

- classic: 
1. no external support, simple error;
2. no external support, robust error;
3. no external support, clusterd by legislature;
4. external support, simple error;
5. external support, robust error;
6. external support, clusterd by legislature

- ratio:
7. no external support, simple error;
8. no external support, robust error;
9. no external support, clusterd by legislature;
10. external support, simple error;
11. external support, robust error;
12. external support, clusterd by legislature

- logit:
13. no external support, simple error;
14. no external support, robust error;
15. no external support, clusterd by legislature;
16. external support, simple error;
17. external support, robust error;
18. external support, clusterd by legislature


## Classic

https://data.princeton.edu/wws509/r/robust

### No external support, simple error;

In [4]:
c1 <- lm(perc_of_populist_toks ~ left_right + classic_gal_tan + Gov_memb + factor(year), data = grouped_df)

summary(c1)


Call:
lm(formula = perc_of_populist_toks ~ left_right + classic_gal_tan + 
    Gov_memb + factor(year), data = grouped_df)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0047273 -0.0004554 -0.0000329  0.0003539  0.0145809 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       2.832e-03  3.964e-04   7.143 2.46e-12 ***
left_right        9.847e-07  1.988e-06   0.495 0.620455    
classic_gal_tan   5.515e-05  1.462e-05   3.772 0.000177 ***
Gov_memb         -3.074e-04  1.018e-04  -3.019 0.002637 ** 
factor(year)1949 -4.008e-04  5.546e-04  -0.723 0.470182    
factor(year)1950  6.593e-05  5.548e-04   0.119 0.905439    
factor(year)1951  2.253e-04  5.555e-04   0.406 0.685188    
factor(year)1952  5.752e-04  5.555e-04   1.035 0.300842    
factor(year)1953  2.898e-04  5.302e-04   0.547 0.584868    
factor(year)1954  2.367e-04  5.732e-04   0.413 0.679728    
factor(year)1955  4.474e-04  5.746e-04   0.779 0.436502    
factor(year)1956 

### No external support; robust error

In [5]:
c2 <- coeftest(c1, vcov = vcovHC(c1, type="HC1"))

c2


t test of coefficients:

                    Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)       2.8317e-03  2.4872e-04 11.3848 < 2.2e-16 ***
left_right        9.8467e-07  1.6609e-06  0.5929 0.5534751    
classic_gal_tan   5.5148e-05  1.4225e-05  3.8767 0.0001167 ***
Gov_memb         -3.0743e-04  1.0944e-04 -2.8090 0.0051194 ** 
factor(year)1949 -4.0077e-04  2.9928e-04 -1.3391 0.1810056    
factor(year)1950  6.5931e-05  3.1451e-04  0.2096 0.8340226    
factor(year)1951  2.2529e-04  3.3587e-04  0.6708 0.5025970    
factor(year)1952  5.7519e-04  3.3534e-04  1.7153 0.0867768 .  
factor(year)1953  2.8978e-04  3.7074e-04  0.7816 0.4347161    
factor(year)1954  2.3673e-04  3.0007e-04  0.7889 0.4304428    
factor(year)1955  4.4738e-04  3.2379e-04  1.3817 0.1675449    
factor(year)1956  3.7105e-04  2.6770e-04  1.3861 0.1662025    
factor(year)1957 -3.3005e-05  2.9121e-04 -0.1133 0.9097967    
factor(year)1958 -2.1296e-05  2.8227e-04 -0.0754 0.9398841    
factor(year)1959  2.0690e-04 

### No external support, clustered by legislature

In [6]:
c3 <- coeftest(c1, vcov = vcovCL, cluster = ~legislature)

c3


t test of coefficients:

                    Estimate  Std. Error  t value  Pr(>|t|)    
(Intercept)       2.8317e-03  8.4366e-05  33.5641 < 2.2e-16 ***
left_right        9.8467e-07  2.6004e-06   0.3787 0.7050672    
classic_gal_tan   5.5148e-05  1.8419e-05   2.9940 0.0028581 ** 
Gov_memb         -3.0743e-04  1.4728e-04  -2.0874 0.0372434 *  
factor(year)1949 -4.0077e-04  1.6364e-05 -24.4904 < 2.2e-16 ***
factor(year)1950  6.5931e-05  2.5786e-05   2.5569 0.0107890 *  
factor(year)1951  2.2529e-04  4.6014e-05   4.8962 1.235e-06 ***
factor(year)1952  5.7519e-04  4.6014e-05  12.5004 < 2.2e-16 ***
factor(year)1953  2.8978e-04  9.2478e-05   3.1335 0.0018051 ** 
factor(year)1954  2.3673e-04  6.1512e-05   3.8486 0.0001306 ***
factor(year)1955  4.4738e-04  8.7305e-05   5.1244 3.947e-07 ***
factor(year)1956  3.7105e-04  8.7305e-05   4.2501 2.451e-05 ***
factor(year)1957 -3.3005e-05  6.5251e-05  -0.5058 0.6131526    
factor(year)1958 -2.1296e-05  1.2527e-04  -0.1700 0.8650618    
factor(year)19

### External support, simple error

In [7]:
c4 <- lm(perc_of_populist_toks ~ left_right + classic_gal_tan + Gov_memb + External_sup + factor(year), data = grouped_df)

summary(c4)


Call:
lm(formula = perc_of_populist_toks ~ left_right + classic_gal_tan + 
    Gov_memb + External_sup + factor(year), data = grouped_df)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0047180 -0.0004598 -0.0000330  0.0003587  0.0145715 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       2.822e-03  3.963e-04   7.123 2.83e-12 ***
left_right        8.324e-07  1.990e-06   0.418 0.675859    
classic_gal_tan   5.536e-05  1.461e-05   3.788 0.000166 ***
Gov_memb         -2.914e-04  1.025e-04  -2.842 0.004625 ** 
External_sup      2.612e-04  2.006e-04   1.302 0.193379    
factor(year)1949 -3.990e-04  5.543e-04  -0.720 0.471923    
factor(year)1950  6.853e-05  5.545e-04   0.124 0.901672    
factor(year)1951  2.297e-04  5.552e-04   0.414 0.679202    
factor(year)1952  5.796e-04  5.552e-04   1.044 0.296900    
factor(year)1953  2.953e-04  5.299e-04   0.557 0.577579    
factor(year)1954  2.431e-04  5.729e-04   0.424 0.671422    
fa

### External support, robust error

In [8]:
c5 <- coeftest(c4, vcov = vcovHC(c4, type="HC1"))

c5


t test of coefficients:

                    Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)       2.8224e-03  2.4981e-04 11.2982 < 2.2e-16 ***
left_right        8.3237e-07  1.6872e-06  0.4934 0.6219267    
classic_gal_tan   5.5358e-05  1.4305e-05  3.8699 0.0001200 ***
Gov_memb         -2.9138e-04  1.1029e-04 -2.6419 0.0084436 ** 
External_sup      2.6123e-04  2.6092e-04  1.0012 0.3171162    
factor(year)1949 -3.9899e-04  3.0007e-04 -1.3297 0.1840971    
factor(year)1950  6.8534e-05  3.1464e-04  0.2178 0.8276405    
factor(year)1951  2.2971e-04  3.3597e-04  0.6837 0.4944011    
factor(year)1952  5.7960e-04  3.3670e-04  1.7214 0.0856512 .  
factor(year)1953  2.9528e-04  3.7184e-04  0.7941 0.4274239    
factor(year)1954  2.4313e-04  3.0066e-04  0.8087 0.4190081    
factor(year)1955  4.5081e-04  3.2581e-04  1.3837 0.1669370    
factor(year)1956  3.7448e-04  2.6858e-04  1.3943 0.1637031    
factor(year)1957 -9.1764e-05  3.0666e-04 -0.2992 0.7648534    
factor(year)1958 -7.2145e-05 

### External support, clustered by legislature

In [9]:
c6 <- coeftest(c4, vcov = vcovCL, cluster = ~legislature)

c6


t test of coefficients:

                    Estimate  Std. Error  t value  Pr(>|t|)    
(Intercept)       2.8224e-03  8.7181e-05  32.3744 < 2.2e-16 ***
left_right        8.3237e-07  2.6676e-06   0.3120 0.7551229    
classic_gal_tan   5.5358e-05  1.8386e-05   3.0108 0.0027071 ** 
Gov_memb         -2.9138e-04  1.5060e-04  -1.9348 0.0534570 .  
External_sup      2.6123e-04  3.5553e-04   0.7348 0.4627536    
factor(year)1949 -3.9899e-04  1.6734e-05 -23.8434 < 2.2e-16 ***
factor(year)1950  6.8534e-05  2.6669e-05   2.5698 0.0104000 *  
factor(year)1951  2.2971e-04  4.7468e-05   4.8393 1.632e-06 ***
factor(year)1952  5.7960e-04  4.7468e-05  12.2104 < 2.2e-16 ***
factor(year)1953  2.9528e-04  9.4388e-05   3.1283 0.0018370 ** 
factor(year)1954  2.4313e-04  6.3572e-05   3.8245 0.0001438 ***
factor(year)1955  4.5081e-04  8.8098e-05   5.1172 4.095e-07 ***
factor(year)1956  3.7448e-04  8.8098e-05   4.2508 2.445e-05 ***
factor(year)1957 -9.1764e-05  9.4869e-05  -0.9673 0.3337720    
factor(year)19

## Ratio

### No external support, simple error

In [10]:
c7 <- lm(perc_of_populist_toks ~ ratio_leftright + ratio_gal_tan + Gov_memb + factor(year), data = grouped_df)

summary(c7)


Call:
lm(formula = perc_of_populist_toks ~ ratio_leftright + ratio_gal_tan + 
    Gov_memb + factor(year), data = grouped_df)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0048268 -0.0004383 -0.0000428  0.0003839  0.0146866 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       2.565e-03  6.190e-04   4.144 3.97e-05 ***
ratio_leftright  -5.086e-06  9.046e-05  -0.056 0.955187    
ratio_gal_tan     1.654e-04  9.728e-05   1.700 0.089634 .  
Gov_memb         -3.249e-04  1.142e-04  -2.846 0.004601 ** 
factor(year)1949 -4.209e-04  8.394e-04  -0.501 0.616295    
factor(year)1950 -3.152e-05  9.067e-04  -0.035 0.972285    
factor(year)1951  5.030e-04  7.583e-04   0.663 0.507366    
factor(year)1952  8.506e-04  7.583e-04   1.122 0.262455    
factor(year)1953  5.141e-04  7.308e-04   0.703 0.482108    
factor(year)1954  5.225e-04  7.477e-04   0.699 0.484943    
factor(year)1955  6.021e-04  7.631e-04   0.789 0.430407    
factor(year)19

### No external support, robust error

In [11]:
c8 <- coeftest(c7, vcov = vcovHC(c7, type="HC1"))

c8


t test of coefficients:

                    Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)       2.5649e-03  4.7114e-04  5.4439 7.927e-08 ***
ratio_leftright  -5.0858e-06  8.6907e-05 -0.0585 0.9533559    
ratio_gal_tan     1.6541e-04  9.7087e-05  1.7038 0.0890006 .  
Gov_memb         -3.2490e-04  1.1347e-04 -2.8634 0.0043544 ** 
factor(year)1949 -4.2088e-04  5.1324e-04 -0.8200 0.4125554    
factor(year)1950 -3.1516e-05  6.6922e-04 -0.0471 0.9624560    
factor(year)1951  5.0304e-04  4.8704e-04  1.0328 0.3021380    
factor(year)1952  8.5063e-04  5.3024e-04  1.6042 0.1092496    
factor(year)1953  5.1408e-04  5.4734e-04  0.9392 0.3480288    
factor(year)1954  5.2251e-04  4.9395e-04  1.0578 0.2906137    
factor(year)1955  6.0214e-04  5.3540e-04  1.1247 0.2612378    
factor(year)1956  5.6816e-04  4.8055e-04  1.1823 0.2376027    
factor(year)1957  2.0126e-04  4.9967e-04  0.4028 0.6872619    
factor(year)1958  2.2832e-04  4.9153e-04  0.4645 0.6424612    
factor(year)1959  5.0396e-04 

### No external support, clustered by legislature

In [12]:
c9 <- coeftest(c7, vcov = vcovCL, cluster = ~legislature)

c9


t test of coefficients:

                    Estimate  Std. Error  t value  Pr(>|t|)    
(Intercept)       2.5649e-03  2.5136e-04  10.2037 < 2.2e-16 ***
ratio_leftright  -5.0858e-06  1.2345e-04  -0.0412 0.9671537    
ratio_gal_tan     1.6541e-04  1.5310e-04   1.0804 0.2804377    
Gov_memb         -3.2490e-04  1.4632e-04  -2.2204 0.0268051 *  
factor(year)1949 -4.2088e-04  3.6581e-05 -11.5054 < 2.2e-16 ***
factor(year)1950 -3.1516e-05  5.8411e-05  -0.5396 0.5897271    
factor(year)1951  5.0304e-04  1.9170e-04   2.6241 0.0089334 ** 
factor(year)1952  8.5063e-04  1.9170e-04   4.4374 1.105e-05 ***
factor(year)1953  5.1408e-04  2.6413e-04   1.9463 0.0521329 .  
factor(year)1954  5.2251e-04  2.3098e-04   2.2622 0.0240854 *  
factor(year)1955  6.0214e-04  2.1660e-04   2.7800 0.0056263 ** 
factor(year)1956  5.6816e-04  2.1660e-04   2.6231 0.0089604 ** 
factor(year)1957  2.0126e-04  2.3257e-04   0.8654 0.3872055    
factor(year)1958  2.2832e-04  2.7426e-04   0.8325 0.4054957    
factor(year)19

### External support, simple error

In [13]:
c10 <- lm(perc_of_populist_toks ~ ratio_leftright + ratio_gal_tan + Gov_memb + External_sup + factor(year), data = grouped_df)

summary(c10)


Call:
lm(formula = perc_of_populist_toks ~ ratio_leftright + ratio_gal_tan + 
    Gov_memb + External_sup + factor(year), data = grouped_df)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0048301 -0.0004622 -0.0000509  0.0003781  0.0146805 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       2.530e-03  6.172e-04   4.099 4.79e-05 ***
ratio_leftright  -1.962e-05  9.043e-05  -0.217 0.828338    
ratio_gal_tan     1.921e-04  9.778e-05   1.965 0.049934 *  
Gov_memb         -2.919e-04  1.149e-04  -2.541 0.011331 *  
External_sup      4.880e-04  2.298e-04   2.123 0.034179 *  
factor(year)1949 -4.126e-04  8.367e-04  -0.493 0.622104    
factor(year)1950 -2.582e-05  9.038e-04  -0.029 0.977221    
factor(year)1951  5.454e-04  7.561e-04   0.721 0.471040    
factor(year)1952  8.930e-04  7.561e-04   1.181 0.238117    
factor(year)1953  5.599e-04  7.288e-04   0.768 0.442642    
factor(year)1954  5.636e-04  7.455e-04   0.756 0.449991    

### External support, robust error

In [14]:
c11 <- coeftest(c10, vcov = vcovHC(c10, type="HC1"))

c11


t test of coefficients:

                    Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)       2.5298e-03  4.7159e-04  5.3644 1.208e-07 ***
ratio_leftright  -1.9618e-05  8.8107e-05 -0.2227 0.8238856    
ratio_gal_tan     1.9213e-04  9.7294e-05  1.9747 0.0488109 *  
Gov_memb         -2.9188e-04  1.1339e-04 -2.5741 0.0103161 *  
External_sup      4.8802e-04  3.2855e-04  1.4854 0.1380257    
factor(year)1949 -4.1263e-04  5.1673e-04 -0.7985 0.4249133    
factor(year)1950 -2.5817e-05  6.7448e-04 -0.0383 0.9694813    
factor(year)1951  5.4537e-04  4.8749e-04  1.1187 0.2637591    
factor(year)1952  8.9296e-04  5.3164e-04  1.6796 0.0936082 .  
factor(year)1953  5.5994e-04  5.4861e-04  1.0206 0.3078805    
factor(year)1954  5.6357e-04  4.9370e-04  1.1415 0.2541574    
factor(year)1955  6.4281e-04  5.3784e-04  1.1952 0.2325515    
factor(year)1956  6.0883e-04  4.8104e-04  1.2656 0.2061897    
factor(year)1957  1.6313e-04  5.1547e-04  0.3165 0.7517611    
factor(year)1958  2.0616e-04 

### External support, clustered by legislature

In [15]:
c12 <- coeftest(c10, vcov = vcovCL, cluster = ~legislature)

c12


t test of coefficients:

                    Estimate  Std. Error  t value  Pr(>|t|)    
(Intercept)       2.5298e-03  2.5789e-04   9.8097 < 2.2e-16 ***
ratio_leftright  -1.9618e-05  1.2647e-04  -0.1551 0.8767851    
ratio_gal_tan     1.9213e-04  1.5027e-04   1.2785 0.2016165    
Gov_memb         -2.9188e-04  1.4685e-04  -1.9876 0.0473657 *  
External_sup      4.8802e-04  4.7042e-04   1.0374 0.3000072    
factor(year)1949 -4.1263e-04  3.6713e-05 -11.2392 < 2.2e-16 ***
factor(year)1950 -2.5817e-05  5.7912e-05  -0.4458 0.6559243    
factor(year)1951  5.4537e-04  1.9143e-04   2.8490 0.0045542 ** 
factor(year)1952  8.9296e-04  1.9143e-04   4.6648 3.903e-06 ***
factor(year)1953  5.5994e-04  2.6695e-04   2.0976 0.0364087 *  
factor(year)1954  5.6357e-04  2.3631e-04   2.3849 0.0174275 *  
factor(year)1955  6.4281e-04  2.1851e-04   2.9417 0.0034042 ** 
factor(year)1956  6.0883e-04  2.1851e-04   2.7862 0.0055206 ** 
factor(year)1957  1.6313e-04  2.3416e-04   0.6967 0.4863087    
factor(year)19

## Logit

### No external support, simple error

In [16]:
c13 <- lm(perc_of_populist_toks ~ logit_left_right + logit_gal_tan + Gov_memb + factor(year), data = grouped_df)

summary(c13)


Call:
lm(formula = perc_of_populist_toks ~ logit_left_right + logit_gal_tan + 
    Gov_memb + factor(year), data = grouped_df)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0046829 -0.0004383 -0.0000277  0.0003816  0.0146026 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       2.765e-03  3.982e-04   6.944 9.28e-12 ***
logit_left_right -2.122e-05  3.158e-05  -0.672 0.501782    
logit_gal_tan     1.461e-04  4.363e-05   3.350 0.000856 ***
Gov_memb         -2.643e-04  1.041e-04  -2.539 0.011352 *  
factor(year)1949 -3.960e-04  5.560e-04  -0.712 0.476584    
factor(year)1950  9.842e-05  5.562e-04   0.177 0.859596    
factor(year)1951  2.849e-04  5.577e-04   0.511 0.609665    
factor(year)1952  6.348e-04  5.577e-04   1.138 0.255447    
factor(year)1953  3.693e-04  5.327e-04   0.693 0.488361    
factor(year)1954  3.119e-04  5.760e-04   0.542 0.588331    
factor(year)1955  5.068e-04  5.778e-04   0.877 0.380802    
factor(year)1

### No external support, robust error

In [17]:
c14 <- coeftest(c13, vcov = vcovHC(c13, type="HC1"))

c14


t test of coefficients:

                    Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)       2.7650e-03  2.5420e-04 10.8775 < 2.2e-16 ***
logit_left_right -2.1222e-05  3.1696e-05 -0.6696 0.5033784    
logit_gal_tan     1.4612e-04  3.8614e-05  3.7843 0.0001685 ***
Gov_memb         -2.6428e-04  1.1390e-04 -2.3202 0.0206380 *  
factor(year)1949 -3.9598e-04  3.0657e-04 -1.2916 0.1969460    
factor(year)1950  9.8417e-05  3.1379e-04  0.3136 0.7538946    
factor(year)1951  2.8486e-04  3.3171e-04  0.8587 0.3908010    
factor(year)1952  6.3475e-04  3.3473e-04  1.8963 0.0583659 .  
factor(year)1953  3.6934e-04  3.6949e-04  0.9996 0.3178831    
factor(year)1954  3.1193e-04  2.9303e-04  1.0645 0.2874967    
factor(year)1955  5.0679e-04  3.3539e-04  1.5111 0.1312616    
factor(year)1956  4.3046e-04  2.7693e-04  1.5544 0.1205716    
factor(year)1957  5.1101e-05  2.8966e-04  0.1764 0.8600193    
factor(year)1958  7.5122e-05  2.8590e-04  0.2628 0.7928222    
factor(year)1959  3.1261e-04 

### No external support, clusterd by legislature

In [18]:
c15 <- coeftest(c13, vcov = vcovCL, cluster = ~legislature)

c15


t test of coefficients:

                    Estimate  Std. Error  t value  Pr(>|t|)    
(Intercept)       2.7650e-03  9.0051e-05  30.7052 < 2.2e-16 ***
logit_left_right -2.1222e-05  3.8572e-05  -0.5502 0.5823761    
logit_gal_tan     1.4612e-04  5.3730e-05   2.7196 0.0067108 ** 
Gov_memb         -2.6428e-04  1.4443e-04  -1.8299 0.0677316 .  
factor(year)1949 -3.9598e-04  1.6047e-05 -24.6758 < 2.2e-16 ***
factor(year)1950  9.8417e-05  2.3683e-05   4.1557 3.679e-05 ***
factor(year)1951  2.8486e-04  5.3490e-05   5.3254 1.391e-07 ***
factor(year)1952  6.3475e-04  5.3490e-05  11.8667 < 2.2e-16 ***
factor(year)1953  3.6934e-04  1.2376e-04   2.9844 0.0029487 ** 
factor(year)1954  3.1193e-04  8.1969e-05   3.8055 0.0001550 ***
factor(year)1955  5.0679e-04  9.7424e-05   5.2019 2.651e-07 ***
factor(year)1956  4.3046e-04  9.7424e-05   4.4184 1.166e-05 ***
factor(year)1957  5.1101e-05  8.4058e-05   0.6079 0.5434498    
factor(year)1958  7.5122e-05  1.4238e-04   0.5276 0.5979509    
factor(year)19

### External support, simple error

In [19]:
c16 <- lm(perc_of_populist_toks ~ logit_left_right + logit_gal_tan + Gov_memb + External_sup + factor(year), data = grouped_df)

summary(c16)


Call:
lm(formula = perc_of_populist_toks ~ logit_left_right + logit_gal_tan + 
    Gov_memb + External_sup + factor(year), data = grouped_df)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0046671 -0.0004444 -0.0000360  0.0003821  0.0145906 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       2.750e-03  3.978e-04   6.914 1.14e-11 ***
logit_left_right -2.466e-05  3.161e-05  -0.780 0.435540    
logit_gal_tan     1.523e-04  4.374e-05   3.483 0.000530 ***
Gov_memb         -2.408e-04  1.050e-04  -2.294 0.022129 *  
External_sup      3.263e-04  2.018e-04   1.617 0.106284    
factor(year)1949 -3.934e-04  5.553e-04  -0.708 0.478950    
factor(year)1950  1.029e-04  5.555e-04   0.185 0.853159    
factor(year)1951  2.949e-04  5.570e-04   0.529 0.596715    
factor(year)1952  6.448e-04  5.570e-04   1.158 0.247470    
factor(year)1953  3.817e-04  5.321e-04   0.717 0.473389    
factor(year)1954  3.262e-04  5.754e-04   0.567 0.571012   

### External support, robust error

In [20]:
c17 <- coeftest(c16, vcov = vcovHC(c16, type="HC1"))

c17


t test of coefficients:

                    Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)       2.7502e-03  2.5601e-04 10.7424 < 2.2e-16 ***
logit_left_right -2.4663e-05  3.2109e-05 -0.7681 0.4427066    
logit_gal_tan     1.5234e-04  3.8866e-05  3.9196 9.816e-05 ***
Gov_memb         -2.4077e-04  1.1501e-04 -2.0935 0.0366973 *  
External_sup      3.2634e-04  2.6302e-04  1.2407 0.2151621    
factor(year)1949 -3.9337e-04  3.0829e-04 -1.2760 0.2024282    
factor(year)1950  1.0285e-04  3.1430e-04  0.3272 0.7435930    
factor(year)1951  2.9487e-04  3.3193e-04  0.8884 0.3746804    
factor(year)1952  6.4477e-04  3.3673e-04  1.9148 0.0559635 .  
factor(year)1953  3.8173e-04  3.7099e-04  1.0289 0.3038950    
factor(year)1954  3.2615e-04  2.9367e-04  1.1106 0.2671544    
factor(year)1955  5.1991e-04  3.3763e-04  1.5399 0.1240784    
factor(year)1956  4.4359e-04  2.7836e-04  1.5936 0.1115177    
factor(year)1957 -1.5612e-05  3.0713e-04 -0.0508 0.9594745    
factor(year)1958  1.9140e-05 

### External support, clustered by legislature

In [21]:
c18 <- coeftest(c16, vcov = vcovCL, cluster = ~legislature)

c18


t test of coefficients:

                    Estimate  Std. Error  t value  Pr(>|t|)    
(Intercept)       2.7502e-03  9.3055e-05  29.5547 < 2.2e-16 ***
logit_left_right -2.4663e-05  3.9287e-05  -0.6278 0.5303813    
logit_gal_tan     1.5234e-04  5.4513e-05   2.7945 0.0053517 ** 
Gov_memb         -2.4077e-04  1.4841e-04  -1.6223 0.1052185    
External_sup      3.2634e-04  3.4661e-04   0.9415 0.3467987    
factor(year)1949 -3.9337e-04  1.6490e-05 -23.8551 < 2.2e-16 ***
factor(year)1950  1.0285e-04  2.4614e-05   4.1787 3.335e-05 ***
factor(year)1951  2.9487e-04  5.5610e-05   5.3025 1.570e-07 ***
factor(year)1952  6.4477e-04  5.5610e-05  11.5944 < 2.2e-16 ***
factor(year)1953  3.8173e-04  1.2725e-04   2.9998 0.0028051 ** 
factor(year)1954  3.2615e-04  8.4816e-05   3.8454 0.0001323 ***
factor(year)1955  5.1991e-04  9.9217e-05   5.2402 2.175e-07 ***
factor(year)1956  4.4359e-04  9.9217e-05   4.4708 9.202e-06 ***
factor(year)1957 -1.5612e-05  1.0380e-04  -0.1504 0.8804935    
factor(year)19

## Classic economic

https://data.princeton.edu/wws509/r/robust

### No external support, simple error;

In [22]:
c19 <- lm(perc_of_populist_toks ~ classic_economic + classic_gal_tan + Gov_memb + factor(year), data = grouped_df)

summary(c19)


Call:
lm(formula = perc_of_populist_toks ~ classic_economic + classic_gal_tan + 
    Gov_memb + factor(year), data = grouped_df)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0047378 -0.0004474 -0.0000268  0.0003680  0.0145741 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       2.865e-03  3.960e-04   7.233 1.34e-12 ***
classic_economic  4.478e-06  3.346e-06   1.338 0.181321    
classic_gal_tan   5.508e-05  1.460e-05   3.773 0.000176 ***
Gov_memb         -3.097e-04  9.946e-05  -3.114 0.001928 ** 
factor(year)1949 -4.010e-04  5.540e-04  -0.724 0.469369    
factor(year)1950  6.074e-05  5.540e-04   0.110 0.912733    
factor(year)1951  2.227e-04  5.547e-04   0.402 0.688153    
factor(year)1952  5.726e-04  5.547e-04   1.032 0.302311    
factor(year)1953  2.848e-04  5.293e-04   0.538 0.590779    
factor(year)1954  2.517e-04  5.721e-04   0.440 0.660132    
factor(year)1955  4.462e-04  5.735e-04   0.778 0.436807    
factor(year

### No external support; robust error

In [23]:
c20 <- coeftest(c19, vcov = vcovHC(c19, type="HC1"))

c20


t test of coefficients:

                    Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)       2.8646e-03  2.4836e-04 11.5341 < 2.2e-16 ***
classic_economic  4.4781e-06  2.5380e-06  1.7644 0.0781304 .  
classic_gal_tan   5.5084e-05  1.4046e-05  3.9216 9.736e-05 ***
Gov_memb         -3.0970e-04  1.0234e-04 -3.0261 0.0025755 ** 
factor(year)1949 -4.0103e-04  3.0059e-04 -1.3341 0.1826378    
factor(year)1950  6.0743e-05  3.1711e-04  0.1916 0.8481546    
factor(year)1951  2.2274e-04  3.3896e-04  0.6571 0.5113386    
factor(year)1952  5.7263e-04  3.3537e-04  1.7075 0.0882149 .  
factor(year)1953  2.8475e-04  3.7118e-04  0.7672 0.4432663    
factor(year)1954  2.5168e-04  3.0282e-04  0.8311 0.4062127    
factor(year)1955  4.4624e-04  3.2378e-04  1.3782 0.1686073    
factor(year)1956  3.6991e-04  2.7184e-04  1.3608 0.1740608    
factor(year)1957 -4.0099e-05  2.9173e-04 -0.1375 0.8907170    
factor(year)1958 -3.8492e-05  2.8230e-04 -0.1364 0.8915857    
factor(year)1959  2.1922e-04 

### No external support, clustered by legislature

In [24]:
c21 <- coeftest(c19, vcov = vcovCL, cluster = ~legislature)

c21


t test of coefficients:

                    Estimate  Std. Error  t value  Pr(>|t|)    
(Intercept)       2.8646e-03  7.7034e-05  37.1865 < 2.2e-16 ***
classic_economic  4.4781e-06  3.9263e-06   1.1405 0.2544898    
classic_gal_tan   5.5084e-05  1.7767e-05   3.1004 0.0020165 ** 
Gov_memb         -3.0970e-04  1.4409e-04  -2.1493 0.0319802 *  
factor(year)1949 -4.0103e-04  1.6010e-05 -25.0479 < 2.2e-16 ***
factor(year)1950  6.0743e-05  1.9820e-05   3.0647 0.0022695 ** 
factor(year)1951  2.2274e-04  3.8727e-05   5.7516 1.364e-08 ***
factor(year)1952  5.7263e-04  3.8727e-05  14.7866 < 2.2e-16 ***
factor(year)1953  2.8475e-04  8.6500e-05   3.2919 0.0010491 ** 
factor(year)1954  2.5168e-04  4.5806e-05   5.4945 5.639e-08 ***
factor(year)1955  4.4624e-04  7.0550e-05   6.3252 4.720e-10 ***
factor(year)1956  3.6991e-04  7.0550e-05   5.2433 2.139e-07 ***
factor(year)1957 -4.0099e-05  5.4139e-05  -0.7407 0.4591624    
factor(year)1958 -3.8492e-05  1.0892e-04  -0.3534 0.7239104    
factor(year)19

### External support, simple error

In [25]:
c22 <- lm(perc_of_populist_toks ~ classic_economic + classic_gal_tan + Gov_memb + External_sup + factor(year), data = grouped_df)

summary(c22)


Call:
lm(formula = perc_of_populist_toks ~ classic_economic + classic_gal_tan + 
    Gov_memb + External_sup + factor(year), data = grouped_df)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0047302 -0.0004531 -0.0000261  0.0003635  0.0145668 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       2.857e-03  3.959e-04   7.216 1.51e-12 ***
classic_economic  4.330e-06  3.347e-06   1.294 0.196223    
classic_gal_tan   5.527e-05  1.459e-05   3.788 0.000166 ***
Gov_memb         -2.952e-04  1.000e-04  -2.951 0.003283 ** 
External_sup      2.572e-04  2.002e-04   1.285 0.199235    
factor(year)1949 -3.994e-04  5.537e-04  -0.721 0.470931    
factor(year)1950  6.272e-05  5.538e-04   0.113 0.909853    
factor(year)1951  2.264e-04  5.544e-04   0.408 0.683196    
factor(year)1952  5.763e-04  5.544e-04   1.039 0.299021    
factor(year)1953  2.892e-04  5.291e-04   0.547 0.584875    
factor(year)1954  2.562e-04  5.718e-04   0.448 0.654272 

### External support, robust error

In [26]:
c23 <- coeftest(c22, vcov = vcovHC(c22, type="HC1"))

c23


t test of coefficients:

                    Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)       2.8567e-03  2.4925e-04 11.4612 < 2.2e-16 ***
classic_economic  4.3298e-06  2.5450e-06  1.7013 0.0893614 .  
classic_gal_tan   5.5270e-05  1.4115e-05  3.9158 9.968e-05 ***
Gov_memb         -2.9522e-04  1.0266e-04 -2.8756 0.0041652 ** 
External_sup      2.5724e-04  2.5894e-04  0.9934 0.3208738    
factor(year)1949 -3.9942e-04  3.0133e-04 -1.3255 0.1854654    
factor(year)1950  6.2724e-05  3.1730e-04  0.1977 0.8433584    
factor(year)1951  2.2637e-04  3.3924e-04  0.6673 0.5048211    
factor(year)1952  5.7627e-04  3.3683e-04  1.7108 0.0875914 .  
factor(year)1953  2.8915e-04  3.7240e-04  0.7765 0.4377609    
factor(year)1954  2.5620e-04  3.0350e-04  0.8441 0.3989101    
factor(year)1955  4.4828e-04  3.2567e-04  1.3765 0.1691562    
factor(year)1956  3.7195e-04  2.7254e-04  1.3647 0.1728073    
factor(year)1957 -9.9094e-05  3.0640e-04 -0.3234 0.7464891    
factor(year)1958 -8.9676e-05 

### External support, clustered by legislature

In [27]:
c24 <- coeftest(c22, vcov = vcovCL, cluster = ~legislature)

c24


t test of coefficients:

                    Estimate  Std. Error  t value  Pr(>|t|)    
(Intercept)       2.8567e-03  7.8275e-05  36.4951 < 2.2e-16 ***
classic_economic  4.3298e-06  3.9768e-06   1.0888 0.2766639    
classic_gal_tan   5.5270e-05  1.7724e-05   3.1184 0.0018992 ** 
Gov_memb         -2.9522e-04  1.4633e-04  -2.0175 0.0440575 *  
External_sup      2.5724e-04  3.5046e-04   0.7340 0.4632100    
factor(year)1949 -3.9942e-04  1.6259e-05 -24.5656 < 2.2e-16 ***
factor(year)1950  6.2724e-05  2.0184e-05   3.1076 0.0019691 ** 
factor(year)1951  2.2637e-04  3.9640e-05   5.7107 1.717e-08 ***
factor(year)1952  5.7627e-04  3.9640e-05  14.5375 < 2.2e-16 ***
factor(year)1953  2.8915e-04  8.7740e-05   3.2956 0.0010359 ** 
factor(year)1954  2.5620e-04  4.7030e-05   5.4475 7.270e-08 ***
factor(year)1955  4.4828e-04  7.0723e-05   6.3385 4.356e-10 ***
factor(year)1956  3.7195e-04  7.0723e-05   5.2592 1.970e-07 ***
factor(year)1957 -9.9094e-05  9.3617e-05  -1.0585 0.2902232    
factor(year)19

## Ratio economic

### No external support, simple error

In [28]:
c25 <- lm(perc_of_populist_toks ~ ratio_economic + ratio_gal_tan + Gov_memb + factor(year), data = grouped_df)

summary(c25)


Call:
lm(formula = perc_of_populist_toks ~ ratio_economic + ratio_gal_tan + 
    Gov_memb + factor(year), data = grouped_df)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0048988 -0.0004433 -0.0000499  0.0003789  0.0147092 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       2.641e-03  5.992e-04   4.407 1.26e-05 ***
ratio_economic    6.963e-05  8.681e-05   0.802 0.422829    
ratio_gal_tan     1.554e-04  9.746e-05   1.594 0.111450    
Gov_memb         -3.384e-04  1.108e-04  -3.053 0.002376 ** 
factor(year)1949 -4.242e-04  8.389e-04  -0.506 0.613257    
factor(year)1950 -4.151e-05  9.062e-04  -0.046 0.963487    
factor(year)1951  4.626e-04  7.506e-04   0.616 0.537945    
factor(year)1952  8.102e-04  7.506e-04   1.079 0.280881    
factor(year)1953  4.685e-04  7.202e-04   0.651 0.515575    
factor(year)1954  4.792e-04  7.286e-04   0.658 0.511054    
factor(year)1955  5.619e-04  7.486e-04   0.751 0.453154    
factor(year)195

### No external support, robust error

In [29]:
c26 <- coeftest(c25, vcov = vcovHC(c25, type="HC1"))

c26


t test of coefficients:

                    Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)       2.6409e-03  4.6571e-04  5.6709 2.319e-08 ***
ratio_economic    6.9632e-05  9.2177e-05  0.7554 0.4503309    
ratio_gal_tan     1.5539e-04  9.8272e-05  1.5812 0.1144184    
Gov_memb         -3.3836e-04  1.0295e-04 -3.2865 0.0010803 ** 
factor(year)1949 -4.2425e-04  5.1270e-04 -0.8275 0.4083384    
factor(year)1950 -4.1505e-05  6.6527e-04 -0.0624 0.9502764    
factor(year)1951  4.6259e-04  4.9342e-04  0.9375 0.3489181    
factor(year)1952  8.1018e-04  5.3820e-04  1.5053 0.1328217    
factor(year)1953  4.6855e-04  5.4823e-04  0.8546 0.3931259    
factor(year)1954  4.7918e-04  4.9434e-04  0.9693 0.3328173    
factor(year)1955  5.6195e-04  5.4120e-04  1.0383 0.2995770    
factor(year)1956  5.2797e-04  4.8980e-04  1.0779 0.2815495    
factor(year)1957  1.3801e-04  4.9848e-04  0.2769 0.7819957    
factor(year)1958  1.5573e-04  4.9197e-04  0.3165 0.7517141    
factor(year)1959  4.3844e-04 

### No external support, clustered by legislature

In [30]:
c27 <- coeftest(c25, vcov = vcovCL, cluster = ~legislature)

c27


t test of coefficients:

                    Estimate  Std. Error  t value  Pr(>|t|)    
(Intercept)       2.6409e-03  1.4830e-04  17.8084 < 2.2e-16 ***
ratio_economic    6.9632e-05  1.4733e-04   0.4726 0.6366819    
ratio_gal_tan     1.5539e-04  1.5728e-04   0.9879 0.3236223    
Gov_memb         -3.3836e-04  1.3664e-04  -2.4764 0.0135784 *  
factor(year)1949 -4.2425e-04  3.4159e-05 -12.4198 < 2.2e-16 ***
factor(year)1950 -4.1505e-05  6.1811e-05  -0.6715 0.5021971    
factor(year)1951  4.6259e-04  1.7559e-04   2.6345 0.0086677 ** 
factor(year)1952  8.1018e-04  1.7559e-04   4.6141 4.938e-06 ***
factor(year)1953  4.6855e-04  2.2211e-04   2.1095 0.0353586 *  
factor(year)1954  4.7918e-04  9.4602e-05   5.0653 5.610e-07 ***
factor(year)1955  5.6195e-04  1.6030e-04   3.5056 0.0004936 ***
factor(year)1956  5.2797e-04  1.6030e-04   3.2936 0.0010541 ** 
factor(year)1957  1.3801e-04  1.4060e-04   0.9815 0.3267722    
factor(year)1958  1.5573e-04  1.9198e-04   0.8112 0.4176268    
factor(year)19

### External support, simple error

In [31]:
c28 <- lm(perc_of_populist_toks ~ ratio_economic + ratio_gal_tan + Gov_memb + External_sup + factor(year), data = grouped_df)

summary(c28)


Call:
lm(formula = perc_of_populist_toks ~ ratio_economic + ratio_gal_tan + 
    Gov_memb + External_sup + factor(year), data = grouped_df)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0049069 -0.0004550 -0.0000528  0.0003919  0.0147069 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       2.626e-03  5.974e-04   4.396 1.33e-05 ***
ratio_economic    6.126e-05  8.664e-05   0.707 0.479820    
ratio_gal_tan     1.811e-04  9.795e-05   1.849 0.065003 .  
Gov_memb         -3.095e-04  1.113e-04  -2.780 0.005628 ** 
External_sup      4.767e-04  2.293e-04   2.079 0.038115 *  
factor(year)1949 -4.170e-04  8.363e-04  -0.499 0.618223    
factor(year)1950 -3.438e-05  9.035e-04  -0.038 0.969660    
factor(year)1951  4.903e-04  7.484e-04   0.655 0.512662    
factor(year)1952  8.379e-04  7.484e-04   1.120 0.263388    
factor(year)1953  4.974e-04  7.181e-04   0.693 0.488826    
factor(year)1954  4.965e-04  7.265e-04   0.684 0.494584    


### External support, robust error

In [32]:
c29 <- coeftest(c28, vcov = vcovHC(c28, type="HC1"))

c29


t test of coefficients:

                    Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)       2.6261e-03  4.6641e-04  5.6304 2.898e-08 ***
ratio_economic    6.1259e-05  9.0018e-05  0.6805 0.4964705    
ratio_gal_tan     1.8111e-04  9.8138e-05  1.8455 0.0655207 .  
Gov_memb         -3.0954e-04  1.0165e-04 -3.0450 0.0024404 ** 
External_sup      4.7671e-04  3.2398e-04  1.4714 0.1417691    
factor(year)1949 -4.1704e-04  5.1435e-04 -0.8108 0.4178347    
factor(year)1950 -3.4380e-05  6.6604e-04 -0.0516 0.9588523    
factor(year)1951  4.9029e-04  4.9361e-04  0.9933 0.3210256    
factor(year)1952  8.3788e-04  5.4056e-04  1.5500 0.1217214    
factor(year)1953  4.9739e-04  5.4990e-04  0.9045 0.3661272    
factor(year)1954  4.9653e-04  4.9439e-04  1.0043 0.3156688    
factor(year)1955  5.8176e-04  5.4418e-04  1.0691 0.2855222    
factor(year)1956  5.4778e-04  4.9070e-04  1.1163 0.2647810    
factor(year)1957  8.0115e-05  5.1495e-04  0.1556 0.8764242    
factor(year)1958  1.1256e-04 

### External support, clustered by legislature

In [33]:
c30 <- coeftest(c28, vcov = vcovCL, cluster = ~legislature)

c30


t test of coefficients:

                    Estimate  Std. Error  t value  Pr(>|t|)    
(Intercept)       2.6261e-03  1.4120e-04  18.5991 < 2.2e-16 ***
ratio_economic    6.1259e-05  1.4189e-04   0.4317 0.6661170    
ratio_gal_tan     1.8111e-04  1.5387e-04   1.1771 0.2396918    
Gov_memb         -3.0954e-04  1.3415e-04  -2.3073 0.0214142 *  
External_sup      4.7671e-04  4.6242e-04   1.0309 0.3030538    
factor(year)1949 -4.1704e-04  3.3538e-05 -12.4347 < 2.2e-16 ***
factor(year)1950 -3.4380e-05  6.0025e-05  -0.5728 0.5670507    
factor(year)1951  4.9029e-04  1.6777e-04   2.9224 0.0036201 ** 
factor(year)1952  8.3788e-04  1.6777e-04   4.9942 7.999e-07 ***
factor(year)1953  4.9739e-04  2.1571e-04   2.3058 0.0215029 *  
factor(year)1954  4.9653e-04  8.6852e-05   5.7170 1.798e-08 ***
factor(year)1955  5.8176e-04  1.5452e-04   3.7650 0.0001849 ***
factor(year)1956  5.4778e-04  1.5452e-04   3.5450 0.0004268 ***
factor(year)1957  8.0115e-05  1.6781e-04   0.4774 0.6332532    
factor(year)19

## Logit economic

### No external support, simple error

In [34]:
c31 <- lm(perc_of_populist_toks ~ logit_economic + logit_gal_tan + Gov_memb + factor(year), data = grouped_df)

summary(c31)


Call:
lm(formula = perc_of_populist_toks ~ logit_economic + logit_gal_tan + 
    Gov_memb + factor(year), data = grouped_df)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0048003 -0.0004426 -0.0000227  0.0003702  0.0146421 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       2.848e-03  4.007e-04   7.109 3.11e-12 ***
logit_economic    2.480e-05  2.859e-05   0.868 0.385979    
logit_gal_tan     1.387e-04  4.384e-05   3.164 0.001631 ** 
Gov_memb         -2.996e-04  1.024e-04  -2.926 0.003552 ** 
factor(year)1949 -3.999e-04  5.558e-04  -0.719 0.472124    
factor(year)1950  7.451e-05  5.564e-04   0.134 0.893519    
factor(year)1951  2.629e-04  5.577e-04   0.471 0.637583    
factor(year)1952  6.128e-04  5.577e-04   1.099 0.272336    
factor(year)1953  3.363e-04  5.328e-04   0.631 0.528133    
factor(year)1954  2.720e-04  5.750e-04   0.473 0.636323    
factor(year)1955  4.717e-04  5.778e-04   0.816 0.414602    
factor(year)195

### No external support, robust error

In [35]:
c32 <- coeftest(c31, vcov = vcovHC(c31, type="HC1"))

c32


t test of coefficients:

                    Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)       2.8482e-03  2.5486e-04 11.1754 < 2.2e-16 ***
logit_economic    2.4801e-05  2.6223e-05  0.9458 0.3446041    
logit_gal_tan     1.3869e-04  3.9057e-05  3.5509 0.0004118 ***
Gov_memb         -2.9958e-04  1.0608e-04 -2.8240 0.0048890 ** 
factor(year)1949 -3.9990e-04  3.0055e-04 -1.3306 0.1838046    
factor(year)1950  7.4506e-05  3.1575e-04  0.2360 0.8135340    
factor(year)1951  2.6287e-04  3.3822e-04  0.7772 0.4373154    
factor(year)1952  6.1276e-04  3.3629e-04  1.8222 0.0688935 .  
factor(year)1953  3.3633e-04  3.7335e-04  0.9009 0.3679982    
factor(year)1954  2.7201e-04  2.9823e-04  0.9121 0.3620599    
factor(year)1955  4.7170e-04  3.3540e-04  1.4064 0.1600943    
factor(year)1956  3.9537e-04  2.8130e-04  1.4055 0.1603477    
factor(year)1957  4.0748e-06  2.8460e-04  0.0143 0.9885810    
factor(year)1958  1.6809e-05  2.8538e-04  0.0589 0.9530494    
factor(year)1959  2.5815e-04 

### No external support, clusterd by legislature

In [36]:
c33 <- coeftest(c31, vcov = vcovCL, cluster = ~legislature)

c33


t test of coefficients:

                    Estimate  Std. Error  t value  Pr(>|t|)    
(Intercept)       2.8482e-03  9.7153e-05  29.3165 < 2.2e-16 ***
logit_economic    2.4801e-05  3.4020e-05   0.7290 0.4662562    
logit_gal_tan     1.3869e-04  5.5529e-05   2.4975 0.0127535 *  
Gov_memb         -2.9958e-04  1.4207e-04  -2.1087 0.0353521 *  
factor(year)1949 -3.9990e-04  1.5785e-05 -25.3340 < 2.2e-16 ***
factor(year)1950  7.4506e-05  3.6518e-05   2.0402 0.0417318 *  
factor(year)1951  2.6287e-04  6.0288e-05   4.3602 1.511e-05 ***
factor(year)1952  6.1276e-04  6.0288e-05  10.1640 < 2.2e-16 ***
factor(year)1953  3.3633e-04  1.1649e-04   2.8872 0.0040172 ** 
factor(year)1954  2.7201e-04  6.2596e-05   4.3456 1.613e-05 ***
factor(year)1955  4.7170e-04  1.0149e-04   4.6479 4.066e-06 ***
factor(year)1956  3.9537e-04  1.0149e-04   3.8958 0.0001081 ***
factor(year)1957  4.0748e-06  7.7432e-05   0.0526 0.9580480    
factor(year)1958  1.6809e-05  1.2963e-04   0.1297 0.8968713    
factor(year)19

### External support, simple error

In [37]:
c34 <- lm(perc_of_populist_toks ~ logit_economic + logit_gal_tan + Gov_memb + External_sup + factor(year), data = grouped_df)

summary(c34)


Call:
lm(formula = perc_of_populist_toks ~ logit_economic + logit_gal_tan + 
    Gov_memb + External_sup + factor(year), data = grouped_df)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0047926 -0.0004550 -0.0000214  0.0003649  0.0146333 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       2.835e-03  4.003e-04   7.080 3.77e-12 ***
logit_economic    2.296e-05  2.858e-05   0.803 0.422191    
logit_gal_tan     1.446e-04  4.396e-05   3.289 0.001062 ** 
Gov_memb         -2.789e-04  1.032e-04  -2.703 0.007051 ** 
External_sup      3.089e-04  2.015e-04   1.533 0.125695    
factor(year)1949 -3.976e-04  5.553e-04  -0.716 0.474219    
factor(year)1950  7.936e-05  5.558e-04   0.143 0.886506    
factor(year)1951  2.725e-04  5.572e-04   0.489 0.624911    
factor(year)1952  6.224e-04  5.572e-04   1.117 0.264374    
factor(year)1953  3.478e-04  5.323e-04   0.653 0.513764    
factor(year)1954  2.828e-04  5.744e-04   0.492 0.622624    


### External support, robust error

In [38]:
c35 <- coeftest(c34, vcov = vcovHC(c34, type="HC1"))

c35


t test of coefficients:

                    Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)       2.8345e-03  2.5591e-04 11.0764 < 2.2e-16 ***
logit_economic    2.2957e-05  2.6057e-05  0.8810 0.3786230    
logit_gal_tan     1.4456e-04  3.9209e-05  3.6869 0.0002461 ***
Gov_memb         -2.7885e-04  1.0627e-04 -2.6240 0.0088956 ** 
External_sup      3.0893e-04  2.6033e-04  1.1867 0.2357811    
factor(year)1949 -3.9760e-04  3.0175e-04 -1.3176 0.1880918    
factor(year)1950  7.9364e-05  3.1604e-04  0.2511 0.8018025    
factor(year)1951  2.7255e-04  3.3853e-04  0.8051 0.4210564    
factor(year)1952  6.2244e-04  3.3812e-04  1.8409 0.0660975 .  
factor(year)1953  3.4781e-04  3.7477e-04  0.9280 0.3537314    
factor(year)1954  2.8284e-04  2.9836e-04  0.9480 0.3434912    
factor(year)1955  4.8345e-04  3.3730e-04  1.4333 0.1522559    
factor(year)1956  4.0712e-04  2.8215e-04  1.4429 0.1495285    
factor(year)1957 -6.0648e-05  3.0059e-04 -0.2018 0.8401677    
factor(year)1958 -3.7645e-05 

### External support, clustered by legislature

In [39]:
c36 <- coeftest(c34, vcov = vcovCL, cluster = ~legislature)

c36


t test of coefficients:

                    Estimate  Std. Error  t value  Pr(>|t|)    
(Intercept)       2.8345e-03  9.6343e-05  29.4216 < 2.2e-16 ***
logit_economic    2.2957e-05  3.3538e-05   0.6845 0.4938931    
logit_gal_tan     1.4456e-04  5.6100e-05   2.5769 0.0101912 *  
Gov_memb         -2.7885e-04  1.4475e-04  -1.9264 0.0544898 .  
External_sup      3.0893e-04  3.4518e-04   0.8950 0.3711252    
factor(year)1949 -3.9760e-04  1.6083e-05 -24.7208 < 2.2e-16 ***
factor(year)1950  7.9364e-05  3.6123e-05   2.1970 0.0283723 *  
factor(year)1951  2.7255e-04  6.1119e-05   4.4593 9.697e-06 ***
factor(year)1952  6.2244e-04  6.1119e-05  10.1840 < 2.2e-16 ***
factor(year)1953  3.4781e-04  1.1804e-04   2.9466 0.0033283 ** 
factor(year)1954  2.8284e-04  6.3844e-05   4.4302 1.106e-05 ***
factor(year)1955  4.8345e-04  1.0191e-04   4.7441 2.580e-06 ***
factor(year)1956  4.0712e-04  1.0191e-04   3.9951 7.212e-05 ***
factor(year)1957 -6.0648e-05  1.0710e-04  -0.5663 0.5714022    
factor(year)19

# Stargazer

In [43]:
stargazer(c1, c2, c3, c4, c5, c6, type="text",  title="Results", align = TRUE)


Results
                                                        Dependent variable:                                    
                    -------------------------------------------------------------------------------------------
                     perc_of_populist_toks                         perc_of_populist_toks                       
                              OLS                coefficient                OLS                coefficient     
                                                    test                                          test         
                              (1)              (2)        (3)               (4)              (5)        (6)    
---------------------------------------------------------------------------------------------------------------
left_right                  0.00000          0.00000    0.00000           0.00000          0.00000    0.00000  
                           (0.00000)        (0.00000)  (0.00000)         (0.00000)        (0.00